In [11]:
import os
import glob
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [12]:
algoritmos = [
    "ACO",
    "ES_SA",
    "ES_VND",
]

In [ ]:
# Arquivos que serão usados no TCC

instancias = [
    "Bur26a",
    "Chr12a",
    "Els19",
    "Esc16a",
    "Esc32a",
    "Had12",
    "Kra30a",
    "Lipa20a",
    "Tai256c",
    "Wil50"
]


In [ ]:
pasta = "."

arquivos = glob.glob(os.path.join(pasta, "iteracoes_*.csv"))

dados = {}

tempos_alvo = np.arange(0, 601, 120)

for arquivo in arquivos:
    nome = os.path.basename(arquivo)
    partes = nome.replace(".csv", "").split("_")
    instancia = partes[1]
    algoritmo = "_".join(partes[2:])  # Ex: ES_SA, ACO, ES_VND

    df = pd.read_csv(arquivo)
    if "tempo_decorrido" in df.columns:
        tempo_col = "tempo_decorrido"
    elif "tempo_decorrido_s" in df.columns:
        tempo_col = "tempo_decorrido_s"
    else:
        raise KeyError(f"Coluna de tempo não encontrada em {arquivo}")

    df = df.sort_values(tempo_col)

    amostrados = []
    for t in tempos_alvo:
        idx = (df[tempo_col] - t).abs().idxmin()
        valor = df.loc[idx, "melhor_custo"]
        amostrados.append(valor)

    while len(amostrados) < len(tempos_alvo):
        amostrados.append(amostrados[-1])

    df_amostrado = pd.DataFrame({
        "tempo_decorrido": tempos_alvo,
        "melhor_custo": amostrados
    })

    if instancia not in dados:
        dados[instancia] = {}
    dados[instancia][algoritmo] = df_amostrado

for instancia, algoritmos in dados.items():
    plt.figure(figsize=(8, 5))
    for algoritmo, df in algoritmos.items():
        plt.plot(df["tempo_decorrido"], df["melhor_custo"], marker="o", label=algoritmo)
    plt.title(f"Evolução da Solução - {instancia}")
    plt.xlabel("Tempo decorrido (s)")
    plt.ylabel("Melhor custo")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(f"evolucao_{instancia}.png", dpi=300)
    plt.close()
